# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

**Цель исследования** — построение предсказательной модели по выбору тарифа для конкретного клиента по используемым услугам с значением *accuracy* более 0.75.

**Ход исследования**

Данные о пользователях и используемых услугах находятся в файле `users_behavior.csv`. Заранее известно, что предобработка данных уже сделана, но обзор данных все равно необходисо произвести. 

На этапе анализа данных необходимо будет разделить выборку на обучающую, валидационную и тестовую выборки, исследовать качество разных моделей, меняя гиперпараметры и проверить качество модели уже на тестовой выборке. В качестве дополнительного задания необходимо проверить модель на вменяемость. 

Выполнить проверку гипотез.
 
Таким образом, исследование пройдёт в пять этапов:
 1. Обзор данных;
 2. Исследовательский анализ данных и выбор лучшей модели;
 3. Проверка модели на вменяемость.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
users = pd.read_csv('/datasets/users_behavior.csv')
display(users.head(10))
display(users.info())
display(users.describe())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Таблица **users**:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод**

Данные уже предобработаны, пропусков нет, аномалии также не замечены.

## Разбейте данные на выборки

Дважды применим функцию train_test_split, чтобы получить тренировочную, валидационную и тестовую выборку в соотношении 0.6:0.2:0.2

In [3]:
users_train, users_q = train_test_split(users, test_size=0.4, random_state=8)
users_valid, users_test = train_test_split(users_q, test_size=0.5, random_state=8)
print('Величина тренировочной выборки:', users_train.shape[0])
print('Величина валидационной выборки:', users_valid.shape[0])
print('Величина тестовой выборки:', users_test.shape[0])

Величина тренировочной выборки: 1928
Величина валидационной выборки: 643
Величина тестовой выборки: 643


Разделим данные на признаки и целевые признаки

In [4]:
train_features = users_train.drop('is_ultra', axis = 1)
train_target = users_train['is_ultra']

valid_features = users_valid.drop('is_ultra', axis = 1)
valid_target = users_valid['is_ultra']

test_features = users_test.drop('is_ultra', axis = 1)
test_target = users_test['is_ultra']

**Вывод**

Данные разделены на тренировочную, валидационную и тестовую выборку в соотношении 0.6:0.2:0.2. Каждая выборка разделена на признаки и целевой признак.

## Исследуйте модели

Начнем с модели "решающее дерево". Изменяя гиперпараметр "max_depth" с 1 до 50 определим наилучшую модель.

In [5]:
best_accuracy = 0
best_max_depth = 0
for i in range(1, 51):
    d_tree_model = DecisionTreeClassifier(max_depth = i, random_state = 8)
    d_tree_model.fit(train_features, train_target)
    prediction_valid = d_tree_model.predict(valid_features)
    accuracy = accuracy_score(valid_target, prediction_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = i
print(f'Максимальное значение accuracy для модели DecisionTree на валидационной выборке = {best_accuracy} с max_depth = {best_max_depth}')

Максимальное значение accuracy для модели DecisionTree на валидационной выборке = 0.8133748055987559 с max_depth = 5


Запомним соответствующие гиперпараметры для модели DecisionTree

Продолжим моделью RandomForest. Здесь будем изменять два гипермараметра - это max_depth (максимальная глубина для каждого дерева) и n_estimators (число деревьев).

In [6]:
best_est = 0
best_depth = 0
best_accuracy = 0
for est in range(1, 51):
    for depth in range (1, 15):
        R_forest_model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 8)
        R_forest_model.fit(train_features, train_target)
        predictions_valid = R_forest_model.predict(valid_features)
        accuracy = accuracy_score(valid_target, predictions_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
print(f'Максимальное значение accuracy для модели RandomForest на валидационной выборке = {best_accuracy} с max_depth = {best_depth} и n_estimators = {best_est}')

Максимальное значение accuracy для модели RandomForest на валидационной выборке = 0.8304821150855366 с max_depth = 10 и n_estimators = 9


Модель LogisticRegression не имеет гиперпараметров, поэтому её можно проверять сразу на тестовой выборке. Стоит отметить, что для этой модели лучше делить данные на тренировочную и тестовую выборку в соотношении 0.75:0.25, так как модель более устойчива к переобучению и валидационную выборку можно не применять.

**Вывод**

Для моделей определены гиперпараметры, при которых обученные модели дают наилучшие результаты:
- Максимальное значение accuracy для модели DecisionTree на валидационной выборке = 0.8133748055987559 с max_depth = 5
- Максимальное значение accuracy для модели RandomForest на валидационной выборке = 0.8304821150855366 с max_depth = 10 и n_estimators = 9

## Проверьте модели на тестовой выборке

Определим качество моделей на тестовой выборке

In [7]:
d_tree_model = DecisionTreeClassifier(max_depth = 5, random_state = 8)
d_tree_model.fit(train_features, train_target)
prediction_test = d_tree_model.predict(test_features)
accuracy = accuracy_score(test_target, prediction_test)
print(f'Значение accuracy для модели DecisionTree на тестовой выборке = {accuracy}')

Значение accuracy для модели DecisionTree на тестовой выборке = 0.7698289269051322


In [8]:
R_forest_model = RandomForestClassifier(max_depth = 10, n_estimators = 9, random_state = 8)
R_forest_model.fit(train_features, train_target)
prediction_test = R_forest_model.predict(test_features)
accuracy = accuracy_score(test_target, prediction_test)
print(f'Значение accuracy для модели RandomForest на тестовой выборке = {accuracy}')

Значение accuracy для модели RandomForest на тестовой выборке = 0.7916018662519441


Для LogisticRegression разделим данные на две выборки и определим точность

In [9]:
usersl_train, usersl_test = train_test_split(users, test_size = 0.25, random_state = 8)

train_features_L = usersl_train.drop('is_ultra', axis = 1)
train_target_L = usersl_train['is_ultra']
test_features_L = usersl_test.drop('is_ultra', axis = 1)
test_target_L = usersl_test['is_ultra']

logistic_reg_model = LogisticRegression(random_state = 8)
logistic_reg_model.fit(train_features_L, train_target_L)
prediction_test = logistic_reg_model.predict(test_features_L)
accuracy = accuracy_score(test_target_L, prediction_test)
print(f'Значение accuracy для модели LogisticRegression на тестовой выборке = {accuracy}')

Значение accuracy для модели LogisticRegression на тестовой выборке = 0.7400497512437811


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Эта модель не проходит по требованиям accuracy

**Вывод**

Лучшая по качеству (accuracy) модель - это RandomForest (accuracy = 0.790), однако она и самая медленная по скорости работы. Следующая по качеству модель - это DecisionTree с значением accuracy = 0.77 с высокой скоростью работы и на третьем месте - LogisticRegression с значением 0.74, которая не удовлетворяет требованиям задачи, однако не требует почти никаких усилий по обучению (не требуется определение оптимальных гиперпараметров) и отличается высокой скоростью работы. 

Для использования в практических целях предлагается модель RandomForest, поскольку в данном случае качество - это самый важный критерий для бизнеса: чем выше качество, тем больше прибыли приносит продукт.

## (бонус) Проверьте модели на адекватность

Проверку модели на адекватность будем проводить с помощью фиктивного классификатора DummyClassifier, который в процессе обучения не определяет зависимости между параметрами и целевым параметром, а на основе только целевых параметров тренировочной выборки строит предположения о целевом параметре всех остальных данных, которые будут подаваться на вход после обучения. Этот классификатор удобно использовать, например, для несбалансированных данных, когда пользователей одного тарифа больше. Тогда этот классификатор определит наиболее часто встречающийся тариф и в качестве ответа будет всегда давать его. Чтобы оценить модель на адекватность нужно сравнить величины accuracy для настоящих и фиктивных моделей.

In [10]:
dummy_model = DummyClassifier(strategy = 'most_frequent', random_state = 8)
dummy_model.fit(train_features, train_target)
prediction_test = dummy_model.predict(test_features)
print('Величина accuracy для модели DummyClassifier на тестовой выборке', accuracy_score(test_target, prediction_test))

Величина accuracy для модели DummyClassifier на тестовой выборке 0.6920684292379471


Для модели LogisticRegression было использовано другое разделение данных, поэтому посмотрим также на них

In [11]:
dummy_model = DummyClassifier(strategy = 'most_frequent', random_state = 8)
dummy_model.fit(train_features_L, train_target_L)
prediction_test = dummy_model.predict(test_features_L)
print('Величина accuracy для модели DummyClassifier на тестовой выборке', accuracy_score(test_target_L, prediction_test))

Величина accuracy для модели DummyClassifier на тестовой выборке 0.6990049751243781


**Вывод**

Все модели прошли проверку на адекватность так как показали величину accuracy больше, чем фиктивная модель, которая просто определяла наиболее частый целевой признак.